In [1]:
from utils import get_raw_data

In [2]:
train_data, test_data, y_train, G, train_hosts, test_hosts = get_raw_data(encoding="utf-8")

100%|█████████████████████████████████████████████████████████████████████████████████| 560/560 [00:00<00:00, 560976.89it/s]


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
idx_train, idx_valid = train_test_split(np.arange(len(train_data)), stratify=y_train, test_size=.15)
train = [train_data[idx] for idx in idx_train]
valid = [train_data[idx] for idx in idx_valid]

In [28]:
import keras
max_words = 10000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)
tokenize.fit_on_texts(train_data) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train)
x_valid = tokenize.texts_to_matrix(valid)

In [29]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(y_train)
print(labels.shape)
onehot_enc = OneHotEncoder()
labels = onehot_enc.fit_transform(labels.reshape(-1, 1))
train_labels = labels[idx_train]
valid_labels = labels[idx_valid]

(2125,)


In [30]:
num_classes = labels.shape[1]

In [31]:
from keras import models, layers
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 10
drop_ratio = 0.3
# Build the model
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Dropout(drop_ratio))
model.add(layers.Activation('relu'))
model.add(layers.Dense(128, input_shape=(max_words,)))
model.add(layers.Dropout(drop_ratio))
model.add(layers.Activation('relu'))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])
history = model.fit(x_train, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 1625 samples, validate on 181 samples
Epoch 1/10
1625/1625 [==============================] - ETA: 12s - loss: 2.1352 - accuracy: 0.093 - ETA: 8s - loss: 2.3627 - accuracy: 0.140 - ETA: 7s - loss: 2.5032 - accuracy: 0.16 - ETA: 6s - loss: 2.3659 - accuracy: 0.21 - ETA: 5s - loss: 2.2157 - accuracy: 0.25 - ETA: 5s - loss: 2.1322 - accuracy: 0.29 - ETA: 5s - loss: 2.1419 - accuracy: 0.29 - ETA: 5s - loss: 2.1047 - accuracy: 0.30 - ETA: 4s - loss: 2.0746 - accuracy: 0.30 - ETA: 4s - loss: 2.0631 - accuracy: 0.30 - ETA: 4s - loss: 2.0586 - accuracy: 0.30 - ETA: 4s - loss: 2.0311 - accuracy: 0.31 - ETA: 4s - loss: 2.0264 - accuracy: 0.32 - ETA: 4s - loss: 2.0157 - accuracy: 0.32 - ETA: 4s - loss: 1.9896 - accuracy: 0.33 - ETA: 3s - loss: 1.9681 - accuracy: 0.33 - ETA: 3s - loss: 1.9625 - accuracy: 0.33 - ETA: 3s - loss: 1.9419 - accuracy: 0.34 - ETA: 3s - loss: 1.9218 - accuracy: 0.35 - ETA: 3s - loss: 1.9136 - accuracy: 0.35 - ETA: 3s - loss: 1.9024 - accuracy: 0.35 - ETA: 3s - lo

In [32]:
y_pred = model.predict(x_valid)

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(np.argmax(valid_labels, axis=1), np.argmax(y_pred, axis=1))

array([[57,  4, 23,  1,  1,  7,  0,  1],
       [ 5,  8,  9,  0,  0,  2,  0,  7],
       [17,  4, 49,  0,  1, 10,  1,  5],
       [ 5,  1,  2,  4,  1,  0,  0,  1],
       [ 2,  0,  8,  0,  2,  0,  0,  0],
       [ 5,  1,  5,  0,  0, 18,  0,  1],
       [ 3,  0,  3,  0,  0,  0,  1,  0],
       [11,  8,  7,  0,  0,  1,  0, 17]], dtype=int64)

In [35]:
# Implement soft confusion matrix

In [36]:
from utils import loss_function

In [37]:
loss_function(np.argmax(valid_labels, axis=1), y_pred)

2.4404377627173197